# CLEAN STOCK MARKET DATA
## CLEANING AND RESTRUCTURING  - PART3
Clean stock market data and aggregate at annual level for each company<br>
Read - market_data.csv - a huge 4GB file of stock market information for everyday for every traded company<br>
Write - cleaned_market_data.csv - this is cleaned version of all our companies of interest - used by tableau dashboard and used by the model. The data is aggregated to the level of a year<br>
Please do not run this as the input file is huge 4GB

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [ ]:
# note this is a 4GB file ... It will not be available in the repo
df = pd.read_csv('../data/market_data/market_dat.txt', delimiter ="|",low_memory = False)
df.head()

### 1. SELECT FEATURES

In [ ]:
headers_keep = ['co_code','company_name','co_stkdate','bse_opening_price','bse_high_price','bse_low_price',\
                'bse_closing_price','bse_returns','bse_traded_qty','bse_market_cap','bse_pe','bse_pb',\
                'nse_opening_price','nse_high_price','nse_low_price','nse_returns','nse_traded_qty',\
                'nse_traded_val','nse_market_cap','nse_pe','nse_pb','equity_facevalue_on_stkdate',\
                'eps_on_stkdate','equity_bv_on_stkdate']
df = df.loc[:,headers_keep]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# read the date field to a datetime obj
df['co_stkdate'] = pd.to_datetime(df['co_stkdate'], format="%Y%m%d")

In [ ]:
# look at data from 2014 onwards only
df = df.loc[df['co_stkdate'].dt.year>=2014,:]
df.shape

### 2. OPEN COMPANY IDENTITY INFORMATION
restured_identity.csv - this is for mapping

In [ ]:
cid = pd.read_csv('../data/restructure_identity.csv')
cid = cid.drop(columns='Unnamed: 0')
cid.head()

In [ ]:
# select only companies that were at somepoint listed
df = df.loc[df['co_code'].isin(cid['co_code']),:]

In [ ]:
# the shape hasnt changed because only listed companies ever have market data about them.
# this is a good double check as nothing has changed 
df.shape

### 3. AGGREGATE DATA AT THE YEAR LEVEL

In [ ]:
# collapse all datetime to just the year
df['co_stkdate'] = df['co_stkdate'].dt.year

In [ ]:
df.info()

In [ ]:
# compressing to values for just one year
df=df.groupby(['co_code', 'co_stkdate']).max().reset_index()

In [ ]:
df.shape

### 4.  COMBINE NSE AND BSE DATA
There are two stock exchanges in India so I combine their data -> simple rule which is higher

In [ ]:
def combine_cols(new,bse,nse):
    df[new] = np.nan
    
    #if both are not null set new to the higher of the two
    df.loc[(df[nse].notna()) & (df[bse].notna()) & (df[nse]>=df[bse]),new] =\
    df.loc[(df[nse].notna()) & (df[bse].notna()) & (df[nse]>=df[bse]),nse]
    
    df.loc[(df[nse].notna()) & (df[bse].notna()) & (df[nse]<=df[bse]),new] =\
    df.loc[(df[nse].notna()) & (df[bse].notna()) & (df[nse]<=df[bse]),bse]
   
    df.loc[(df[bse].isnull()),new] = df.loc[(df[bse].isnull()),nse]
   
    df.loc[(df[nse].isnull()),new] = df.loc[(df[nse].isnull()),bse]
    df.drop(bse, axis=1, inplace=True)
    df.drop(nse, axis=1, inplace=True)
    

In [ ]:
combine_cols('high_price','bse_high_price','nse_high_price')
combine_cols('stock_returns','bse_returns','nse_returns')
combine_cols('market_cap','bse_market_cap','nse_market_cap')
combine_cols('pe','bse_pe','nse_pe')
combine_cols('pb','bse_pb','nse_pb')
combine_cols('traded_qty','bse_traded_qty','nse_traded_qty')

In [ ]:
df.head()

### 4. FINAL CLEAN AND WRITE

In [ ]:
# headers to keep
headers = ['co_code', 'co_stkdate', 'company_name','equity_facevalue_on_stkdate',
       'eps_on_stkdate', 'equity_bv_on_stkdate', 'high_price', 'stock_returns',
       'market_cap', 'pe', 'pb', 'traded_qty']
df = df.loc[:,headers]

In [ ]:
df.head()

In [ ]:
# save file
df.to_csv('../data/cleaned_market_data.csv')